In [1]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [1]:
documents[0].keys()

NameError: name 'documents' is not defined

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

In [6]:
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/JmfjoxHdSUug387zN66sWw] already exists')

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc, timeout=30)

  0%|          | 0/948 [00:00<?, ?it/s]


BadRequestError: BadRequestError(400, 'illegal_argument_exception', 'failed to parse setting [timeout] with value [30] as a time value: unit is missing or unrecognized')

In [ ]:
query = "How do I execute a command in a running docker container?"

In [ ]:
body = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=body)
response["hits"]["hits"]

In [1]:
body = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=body)
response["hits"]["hits"]

NameError: name 'query' is not defined